In [4]:
import numpy as np
import matplotlib.pyplot as plt
import apogee.tools.read as apread
import pylab as pl
import os


folder = '/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder/'

def calcR(x, pos1=0, pos2=401):
        '''
        Calculates the value of R with the given array x
        :return:  The value of R
        '''        
        ccfCenter = 201
        pos1+= 1
        tempMirror = (x[ccfCenter:pos2])[::-1]
        sigmaA = np.sqrt(1.0 / (2.0 * len(tempMirror)) * np.sum((x[pos1:ccfCenter] - tempMirror)**2))
        return np.max(x) / (np.sqrt(2.0) * sigmaA)

def findInflection(x):
        '''
        Finds the inflection points of the given curve if available.
        :param x:  The curve to find the inflection points of.
        :return: The position of the inflection points (left, right)
        '''
        maxPos = np.argmax(x)
        pos1, pos2 = -1, -1
        for i in range(maxPos, len(x) - 1):
                if (x[i + 1] - x[i] > 0):
                        pos1 = i
                        break
        for i in range(maxPos, 1, -1):
                if (x[i - 1] - x[i] > 0):
                        pos2 = i
                        break
        return pos2, pos1

def getMaxPositions(x, yBufferRange):
        '''
        Gets the positions of both maximums (if two exists) in the given array
        :param x: The array to find both maximums
        :param yBufferRange:  The minimum difference needed between the two maximums
        :return:  The position of both maximums (max1, max2)
        '''
        temp = np.array(x)
        max1 = np.nanargmax(x)
        pos1, pos2 = findInflection(temp)
        temp[pos1:pos2] = np.nan
        temp[np.where(temp < 0.2)] = np.nan

        # If we can find one, record the second maximum
        try:
                max2 = np.nanargmax(temp)
                '''
                # Check to see wListch inflection point is closest to peak 2
                if (np.abs(x[max1] - x[max2]) < yBufferRange):
                        max2 = 'none'
                '''
                if (np.abs(max2 - pos1) < np.abs(max2 - pos2)):
                        # Check if it's witListn the yBufferRange
                        if (np.abs(x[max2] - x[pos1]) < yBufferRange):
                                max2 = 'none'
                elif (np.abs(max2 - pos1) > np.abs(max2 - pos2)):
                        if (np.abs(x[max2] - x[pos2]) < yBufferRange):
                                max2 = 'none'
        except ValueError:
                max2 = 'none'
        
        # Double check that we are returning different positions
        if str(max1) == str(max2):
                max2 = 'none'
        
        return max1, max2

def recordTargets(targets, ranger, filename):
        targetCount = len(targets)

        if not os.path.exists(folder + str(round(ranger, 2)) + '/'):
                os.makedirs(folder + str(round(ranger, 2)) + '/')
        filename = folder + str(round(ranger, 2)) + '/' + filename + '.csv'
        f = open(filename, 'w')

        for i in range(targetCount):
                f.write(str(targets[i][0]) + ',' + str(targets[i][1]) + '\n')
        f.close()

def reportPositions(locationID, apogeeID, ranger, positions):
        '''
        Records the positions of the maximums for each visit
        :param locationID:  The field ID of the target
        :param apogeeID: The apogee ID of the target
        :param ranger: The range used for yBufferRange in getMaxPositions
        :param positions: The positions of the (potentially two) maximums
        '''
        if positions == []:
                return
        
        visitCount = len(positions)

        if not os.path.exists(folder + str(round(ranger, 2)) + '/' + str(locationID) + '/'):
                os.makedirs(folder + str(round(ranger, 2)) + '/' + str(locationID) + '/')
        filename = folder + str(round(ranger, 2)) + '/' + str(locationID) + '/' + str(apogeeID) + '.tbl'
        f = open(filename, 'w')

        
        f.write('visit\tmax1\tmax2\t')
        for i in range(len(positions[0][2])):
                f.write('r'+str(i+1)+'\t\t\t')
        f.write('\n')

        for i in range(visitCount):
                f.write(str(i + 1))
                position = positions[i]
                # record postions of maximums
                f.write('\t\t' + str(position[0]))
                f.write('\t\t' + str(position[1]))
                # record r values
                for val in position[2]:
                        f.write('\t' + str(val))
                f.write('\n')
        f.close()
        
apogeeIDs,locationIDs = np.loadtxt('DR12_SB9_SB2s.lis', usecols=[4,5], unpack=True, dtype=str)
targetCount = len(apogeeIDs)

SB9Targets = []
skippedSB9Targets = []
for i in range(targetCount):
    locationID = locationIDs[i]
    apogeeID = apogeeIDs[i]
    recorded = False
    for k in range(2, 50):
        ranger = k / 100.

        try:
            badheader, header = apread.apStar(locationID, apogeeID, ext=0, dr='13', header=True)
            data = apread.apStar(locationID, apogeeID, ext=9, header=False, dr='13')
        except IOError:
            skippedSB9Targets.append([locationID, apogeeID])
            break
        nvisits = header['NVISITS']

        positions = []
        for visit in range(1, nvisits):
            if (nvisits != 1):
                ccf = data['CCF'][0][1 + visit]
            else:
                ccf = data['CCF'][0]
            max1, max2 = getMaxPositions(ccf, ranger)

            r = []
            for cut in range(20):
                r.append(calcR(ccf, cut*10, (401 - (cut * 10))))
            
            if r < 7.0:
                if recorded is False:
                    recorded = True
                    SB9Targets.append([locationID, apogeeID, "r"])

            if str(max2) != 'none':
                if recorded is False:
                    recorded = True
                    SB9Targets.append([locationID, apogeeID, ranger])

            positions.append([max1, max2, r])
        
        reportPositions(locationID, apogeeID, ranger, positions)

recordTargets(SB9Targets, ranger, 'SB9Targets')
recordTargets(skippedSB9Targets, ranger, 'skippedSB9Targets')


/Users/reynaa/anaconda/envs/yo/lib/python2.7/site-packages/ipykernel/__main__.py:50: RuntimeWarning: invalid value encountered in less


In [23]:
import numpy as np
import matplotlib.pyplot as plt
import apogee.tools.read as apread
import pylab as pl
import os


folder = '/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder/'

def calcR(x, pos1=0, pos2=401):
        '''
        Calculates the value of R with the given array x
        :return:  The value of R
        '''        
        ccfCenter = 201
        pos1+= 1
        tempMirror = (x[ccfCenter:pos2])[::-1]
        sigmaA = np.sqrt(1.0 / (2.0 * len(tempMirror)) * np.sum((x[pos1:ccfCenter] - tempMirror)**2))
        return np.max(x) / (np.sqrt(2.0) * sigmaA)

        
apogeeIDs,locationIDs = np.loadtxt('DR12_SB9_SB2s.lis', usecols=[4,5], unpack=True, dtype=str)
targetCount = len(apogeeIDs)
print(targetCount)
#print(apogeeIDs)

SB9Targets = []
skippedSB9Targets = []
for i in range(targetCount):
    for visit in range(1, nvisits):
        positions = []
        for visit in range(1, nvisits):
            if (nvisits != 1):
                ccf = data['CCF'][0][1 + visit]
            else:
                ccf = data['CCF'][0]
            max1, max2 = getMaxPositions(ccf, ranger)
            r = []
            for cut in range(20):
                r.append(calcR(ccf, cut*10, (401 - (cut * 10))))
            
            if r < 7.0:
                if recorded is False:
                    recorded = True
                    SB9Targets.append([locationID, apogeeID, "r"])
    

print(r)
#print(len(r))

144


/Users/reynaa/anaconda/envs/yo/lib/python2.7/site-packages/ipykernel/__main__.py:50: RuntimeWarning: invalid value encountered in less


[30.479722042428826, 30.233758555639611, 30.354993581835604, 30.815830073944529, 33.188965796265926, 36.058144214157217, 35.643028893615295, 34.577117461976705, 34.531044707245691, 33.490105209006174, 32.434279429128431, 34.075232586817009, 33.013356379006126, 31.497375962568722, 29.3140989611469, 28.927102007134351, 34.515846302643347, 30.778967410167972, 29.845530039365226, 22.406753845683212]
